# How to manage state in subgraphs

For more complex systems, sub-graphs are a useful design principle. Sub-graphs allow you to create and manage different states in different parts of your graph. This allows you build things like [multi-agent teams](https://langchain-ai.github.io/langgraphjs/tutorials/multi_agent/hierarchical_agent_teams/), where each team can track its own separate state.

In this how-to guide we will cover how to manage the persisted state in subgraphs, including how to support human-in-the-loop interaction patterns via interrupts.

## Setup

First we need to install required packages:

```bash
npm install @langchain/langgraph @langchain/core @langchain/openai
```

Next, we need to set API keys for OpenAI (the provider we'll use for this guide):

In [1]:
// process.env.OPENAI_API_KEY = "YOUR_API_KEY";

<div class="admonition tip">
    <p class="admonition-title">Set up <a href="https://smith.langchain.com">LangSmith</a> for LangGraph development</p>
    <p style="padding-top: 5px;">
        Sign up for LangSmith to quickly spot issues and improve the performance of your LangGraph projects. LangSmith lets you use trace data to debug, test, and monitor your LLM apps built with LangGraph — read more about how to get started <a href="https://docs.smith.langchain.com">here</a>. 
    </p>
</div>    

## Define SubGraph

First, let's set up our subgraph. For this, we will create a simple graph that can get the weather for a specific city. We will compile this graph with a [breakpoint](https://langchain-ai.github.io/langgraphjs/how-tos/human_in_the_loop/breakpoints/) before the `weather_node`:

In [2]:
import { z } from "zod";
import { tool } from "@langchain/core/tools";
import { ChatOpenAI } from "@langchain/openai";
import { StateGraph, MessagesAnnotation, Annotation } from "@langchain/langgraph";

const getWeather = tool(async ({ city }) => {
  return `It's sunny in ${city}`;
}, {
  name: "get_weather",
  description: "Get the weather for a specific city",
  schema: z.object({
    city: z.string().describe("A city name")
  })
});

const rawModel = new ChatOpenAI({ model: "gpt-4o-mini" });
const model = rawModel.withStructuredOutput(getWeather);

// Extend the base MessagesAnnotation state with another field
const SubGraphAnnotation = Annotation.Root({
  ...MessagesAnnotation.spec,
  city: Annotation<string>,
});

const modelNode = async (state: typeof SubGraphAnnotation.State) => {
  const result = await model.invoke(state.messages);
  return { city: result.city };
};

const weatherNode = async (state: typeof SubGraphAnnotation.State) => {
  const result = await getWeather.invoke({ city: state.city });
  return {
    messages: [
      {
        role: "assistant",
        content: result,
      }
    ]
  };
};

const subgraph = new StateGraph(SubGraphAnnotation)
  .addNode("modelNode", modelNode)
  .addNode("weatherNode", weatherNode)
  .addEdge("__start__", "modelNode")
  .addEdge("modelNode", "weatherNode")
  .addEdge("weatherNode", "__end__")
  .compile({ interruptBefore: ["weatherNode"] });

## Define Parent Graph

We can now setup the overall graph. This graph will first route to the subgraph if it needs to get the weather, otherwise it will route to a normal LLM.

In [3]:
import { MemorySaver } from "@langchain/langgraph";

const memory = new MemorySaver();

const RouterStateAnnotation = Annotation.Root({
  ...MessagesAnnotation.spec,
  route: Annotation<"weather" | "other">,
});

const routerModel = rawModel.withStructuredOutput(
  z.object({
    route: z.enum(["weather", "other"]).describe("A step that should execute next to based on the currnet input")
  }),
  {
    name: "router"
  }
);

const routerNode = async (state: typeof RouterStateAnnotation.State) => {
  const systemMessage = {
    role: "system",
    content: "Classify the incoming query as either about weather or not.",
  };
  const messages = [systemMessage, ...state.messages]
  const { route } = await routerModel.invoke(messages);
  return { route };
}

const normalLLMNode = async (state: typeof RouterStateAnnotation.State) => {
  const responseMessage = await rawModel.invoke(state.messages);
  return { messages: [responseMessage] };
};

const routeAfterPrediction = async (state: typeof RouterStateAnnotation.State) => {
  if (state.route === "weather") {
    return "weatherGraph";
  } else {
    return "normalLLMNode";
  }
};

const graph = new StateGraph(RouterStateAnnotation)
  .addNode("routerNode", routerNode)
  .addNode("normalLLMNode", normalLLMNode)
  .addNode("weatherGraph", subgraph)
  .addEdge("__start__", "routerNode")
  .addConditionalEdges("routerNode", routeAfterPrediction)
  .addEdge("normalLLMNode", "__end__")
  .addEdge("weatherGraph", "__end__")
  .compile({ checkpointer: memory });

Let's test this out with a normal query to make sure it works as intended!

In [4]:
const config = { configurable: { thread_id: "1" } };

const inputs = { messages: [{ role: "user", content: "hi!" }] };

const stream = await graph.stream(inputs, { ...config, streamMode: "updates" });

for await (const update of stream) {
  console.log(update);
}

{ routerNode: { route: 'other' } }
{
  normalLLMNode: {
    messages: [
      AIMessage {
        "id": "chatcmpl-ABrhXkuNcs4I00aEHsLZmfWSU7iLp",
        "content": "Hello! How can I assist you today?",
        "additional_kwargs": {},
        "response_metadata": {
          "tokenUsage": {
            "completionTokens": 9,
            "promptTokens": 9,
            "totalTokens": 18
          },
          "finish_reason": "stop",
          "system_fingerprint": "fp_f85bea6784"
        },
        "tool_calls": [],
        "invalid_tool_calls": [],
        "usage_metadata": {
          "input_tokens": 9,
          "output_tokens": 9,
          "total_tokens": 18
        }
      }
    ]
  }
}


Great! We didn't ask about the weather, so we got a normal response from the LLM.

## Resuming from breakpoints

Let's now look at what happens with breakpoints. Let's invoke it with a query that should get routed to the weather subgraph where we have the interrupt node.

In [5]:
const config2 = { configurable: { thread_id: "2" } };

const streamWithBreakpoint = await graph.stream({
  messages: [{
    role: "user",
    content: "what's the weather in sf"
  }]
}, { ...config2, streamMode: "updates" });

for await (const update of streamWithBreakpoint) {
  console.log(update);
}

{ routerNode: { route: 'weather' } }


Note that the graph stream doesn't include subgraph events. If we want to stream subgraph events, we can pass `subgraphs: True` in our config and get back subgraph events like so:

In [6]:
const streamWithSubgraphs = await graph.stream({
  messages: [{
    role: "user",
    content: "what's the weather in sf"
  }]
}, { configurable: { thread_id: "3" }, streamMode: "updates", subgraphs: true });

for await (const update of streamWithSubgraphs) {
  console.log(update);
}

[ [], { routerNode: { route: 'weather' } } ]
[
  [ 'weatherGraph:d2578a12-1e6a-57e0-8eff-f5979b16771b' ],
  { modelNode: { city: 'San Francisco' } }
]


This time, we see the format of the streamed updates has changed. It's now a tuple where the first item is a nested array with information about the subgraph and the second is the actual state update. If we get the state now, we can see that it's paused on `weatherGraph` as expected:

In [7]:
const state = await graph.getState({ configurable: { thread_id: "3" } })
state.next

[ 'weatherGraph' ]


If we look at the pending tasks for our current state, we can see that we have one task named `weatherGraph`, which corresponds to the subgraph task.

In [8]:
JSON.stringify(state.tasks, null, 2);

[
  {
    "id": "d2578a12-1e6a-57e0-8eff-f5979b16771b",
    "name": "weatherGraph",
    "path": [
      "__pregel_pull",
      "weatherGraph"
    ],
    "interrupts": [],
    "state": {
      "configurable": {
        "thread_id": "3",
        "checkpoint_ns": "weatherGraph:d2578a12-1e6a-57e0-8eff-f5979b16771b"
      }
    }
  }
]


However since we got the state using the config of the parent graph, we don't have access to the subgraph state. If you look at the `state` value of the task above you will note that it is simply the configuration of the parent graph. If we want to actually populate the subgraph state, we can pass in `subgraphs: True` to the second parameter of `getState` like so:

In [9]:
const stateWithSubgraphs = await graph.getState({ configurable: { thread_id: "3" } }, { subgraphs: true })
JSON.stringify(stateWithSubgraphs.tasks, null, 2)

[
  {
    "id": "d2578a12-1e6a-57e0-8eff-f5979b16771b",
    "name": "weatherGraph",
    "path": [
      "__pregel_pull",
      "weatherGraph"
    ],
    "interrupts": [],
    "state": {
      "values": {
        "messages": [
          {
            "lc": 1,
            "type": "constructor",
            "id": [
              "langchain_core",
              "messages",
              "HumanMessage"
            ],
            "kwargs": {
              "content": "what's the weather in sf",
              "additional_kwargs": {},
              "response_metadata": {}
            }
          }
        ],
        "city": "San Francisco"
      },
      "next": [
        "weatherNode"
      ],
      "tasks": [
        {
          "id": "f876e5d3-4f2a-5ef5-8b92-72947d7c9787",
          "name": "weatherNode",
          "path": [
            "__pregel_pull",
            "weatherNode"
          ],
          "interrupts": []
        }
      ],
      "metadata": {
        "source": "loop",
        "

Now we have access to the subgraph state!

To resume execution, we can just invoke the outer graph as normal:

In [10]:
const resumedStream = await graph.stream(null, {
  configurable: { thread_id: "3" },
  streamMode: "values",
  subgraphs: true,
});

for await (const update of resumedStream) {
  console.log(update);
}

[
  [],
  {
    messages: [
      HumanMessage {
        "content": "what's the weather in sf",
        "additional_kwargs": {},
        "response_metadata": {}
      }
    ],
    route: 'weather'
  }
]
[
  [ 'weatherGraph:d2578a12-1e6a-57e0-8eff-f5979b16771b' ],
  {
    messages: [
      HumanMessage {
        "id": "afff8f12-4f9c-4eed-a541-2605fe3e5bc1",
        "content": "what's the weather in sf",
        "additional_kwargs": {},
        "response_metadata": {}
      }
    ],
    city: 'San Francisco'
  }
]
[
  [ 'weatherGraph:d2578a12-1e6a-57e0-8eff-f5979b16771b' ],
  {
    messages: [
      HumanMessage {
        "id": "afff8f12-4f9c-4eed-a541-2605fe3e5bc1",
        "content": "what's the weather in sf",
        "additional_kwargs": {},
        "response_metadata": {}
      },
      AIMessage {
        "id": "f1a4d5cc-5a32-4864-98ad-b72f3e544b87",
        "content": "It's sunny in San Francisco",
        "additional_kwargs": {},
        "response_metadata": {},
        "tool_cal

### Resuming from specific subgraph node

In the example above, we were replaying from the outer graph - which automatically replayed the subgraph from whatever state it was in previously (paused before the `weatherNode` in our case), but it is also possible to replay from inside a subgraph. In order to do so, we need to get the configuration from the exact subgraph state that we want to replay from.

We can do this by exploring the state history of the subgraph, and selecting the state before `modelNode` - which we can do by filtering on the `.next` parameter.

To get the state history of the subgraph, we need to first pass in the parent graph state before the subgraph:

In [11]:
let parentGraphStateBeforeSubgraph;

const histories = await graph.getStateHistory({ configurable: { thread_id: "3" } });

for await (const historyEntry of histories) {
  if (historyEntry.next[0] === "weatherGraph") {
    parentGraphStateBeforeSubgraph = historyEntry;
  }
}

In [12]:
let subgraphStateBeforeModelNode;

const subgraphHistories = await graph.getStateHistory(parentGraphStateBeforeSubgraph.tasks[0].state);

for await (const subgraphHistoryEntry of subgraphHistories) {
  if (subgraphHistoryEntry.next[0] === "modelNode") {
    subgraphStateBeforeModelNode = subgraphHistoryEntry;
  }
}

console.log(subgraphStateBeforeModelNode);

{
  values: {
    messages: [
      HumanMessage {
        "content": "what's the weather in sf",
        "additional_kwargs": {},
        "response_metadata": {}
      }
    ]
  },
  next: [ 'modelNode' ],
  tasks: [
    {
      id: '38a8b218-0b68-51e7-8c09-2e2d5a18e66b',
      name: 'modelNode',
      path: [Array],
      interrupts: [],
      state: undefined
    }
  ],
  metadata: {
    source: 'loop',
    writes: null,
    step: 0,
    parents: { '': '1ef7c588-0f15-62c0-8001-922833ac0670' }
  },
  config: {
    configurable: {
      thread_id: '3',
      checkpoint_ns: 'weatherGraph:f9f8b515-0c85-58ec-9c7c-0987fc9a2d74',
      checkpoint_id: '1ef7c588-0f17-69d2-8000-db7585cd182b',
      checkpoint_map: [Object]
    }
  },
  createdAt: '2024-09-26T22:41:43.405Z',
  parentConfig: {
    configurable: {
      thread_id: '3',
      checkpoint_ns: 'weatherGraph:f9f8b515-0c85-58ec-9c7c-0987fc9a2d74',
      checkpoint_id: '1ef7c588-0f17-69d1-ffff-06a7c7be86c9'
    }
  }
}


This pattern can be extended no matter how many levels deep.

We can confirm that we have gotten the correct state by comparing the `.next` parameter of the `subgraphStateBeforeModelNode`.

In [13]:
subgraphStateBeforeModelNode.next;

[ 'modelNode' ]


Perfect! We have gotten the correct state snaphshot, and we can now resume from the `modelNode` inside of our subgraph:

In [14]:
const resumeSubgraphStream = await graph.stream(null, {
  ...subgraphStateBeforeModelNode.config,
  streamMode: "updates",
  subgraphs: true
});

for await (const value of resumeSubgraphStream) {
  console.log(value);
}

[
  [ 'weatherGraph:f9f8b515-0c85-58ec-9c7c-0987fc9a2d74' ],
  { modelNode: { city: 'San Francisco' } }
]


We can see that it reruns the `modelNode` and breaks right before the `weatherNode` as expected.

This subsection has shown how you can replay from any node, no matter how deeply nested it is inside your graph - a powerful tool for testing how deterministic your agent is.

## Modifying state

### Update the state of a subgraph

What if we want to modify the state of a subgraph? We can do this similarly to how we [update the state of normal graphs](https://langchain-ai.github.io/langgraphjs/how-tos/time-travel/). We just need to ensure we pass the config of the subgraph to `updateState`. Let's run our graph as before:

In [15]:
const graphStream = await graph.stream({
  messages: [{
    role: "user",
    content: "what's the weather in sf"
  }],
}, {
  configurable: {
    thread_id: "4",
  }
});

for await (const update of graphStream) {
  console.log(update);
}

{ routerNode: { route: 'weather' } }


In [16]:
const outerGraphState = await graph.getState({
  configurable: {
    thread_id: "4",
  }
}, { subgraphs: true })

console.log(outerGraphState.tasks[0].state);

{
  values: {
    messages: [
      HumanMessage {
        "content": "what's the weather in sf",
        "additional_kwargs": {},
        "response_metadata": {}
      }
    ],
    city: 'San Francisco'
  },
  next: [ 'weatherNode' ],
  tasks: [
    {
      id: '0e73ac86-3c63-5424-8f54-98be2a753632',
      name: 'weatherNode',
      path: [Array],
      interrupts: [],
      state: undefined
    }
  ],
  metadata: {
    source: 'loop',
    writes: { modelNode: [Object] },
    step: 1,
    parents: { '': '1ef7c588-238e-6df1-8001-1eaea6a0d11a' }
  },
  config: {
    configurable: {
      thread_id: '4',
      checkpoint_id: '1ef7c588-2989-6a20-8001-992703213fdb',
      checkpoint_ns: 'weatherGraph:e8782cc9-b27e-52e1-9d58-306465722baf',
      checkpoint_map: [Object]
    }
  },
  createdAt: '2024-09-26T22:41:46.178Z',
  parentConfig: {
    configurable: {
      thread_id: '4',
      checkpoint_ns: 'weatherGraph:e8782cc9-b27e-52e1-9d58-306465722baf',
      checkpoint_id: '1ef7c588-2393-6c

In order to update the state of the **inner** graph, we need to pass the config for the **inner** graph, which we can get by accessing calling `state.tasks[0].state.config` - since we interrupted inside the subgraph, the state of the task is just the state of the subgraph.

In [17]:
import type { StateSnapshot } from "@langchain/langgraph";

await graph.updateState((outerGraphState.tasks[0].state as StateSnapshot).config, { city: "la" });

{
  configurable: {
    thread_id: '4',
    checkpoint_ns: 'weatherGraph:e8782cc9-b27e-52e1-9d58-306465722baf',
    checkpoint_id: '1ef7c588-2a80-6370-8002-c716842440f0',
    checkpoint_map: {
      '': '1ef7c588-238e-6df1-8001-1eaea6a0d11a',
      'weatherGraph:e8782cc9-b27e-52e1-9d58-306465722baf': '1ef7c588-2a80-6370-8002-c716842440f0'
    }
  }
}


We can now resume streaming the outer graph (which will resume the subgraph!) and check that we updated our search to use LA instead of SF.

In [18]:
const resumedStreamWithUpdatedState = await graph.stream(null, {
  configurable: {
    thread_id: "4",
  },
  streamMode: "updates",
  subgraphs: true,
})

for await (const update of resumedStreamWithUpdatedState) {
  console.log(JSON.stringify(update, null, 2));
}

[
  [
    "weatherGraph:e8782cc9-b27e-52e1-9d58-306465722baf"
  ],
  {
    "weatherNode": {
      "messages": [
        {
          "role": "assistant",
          "content": "It's sunny in la"
        }
      ]
    }
  }
]
[
  [],
  {
    "weatherGraph": {
      "messages": [
        {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain_core",
            "messages",
            "HumanMessage"
          ],
          "kwargs": {
            "content": "what's the weather in sf",
            "additional_kwargs": {},
            "response_metadata": {}
          }
        },
        {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain_core",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "It's sunny in la",
            "tool_calls": [],
            "invalid_tool_calls": [],
            "additional_kwargs": {},
            "response_metadata": {}
   

Fantastic! The AI responded with "It's sunny in LA!" as we expected.

### Acting as a subgraph node

Instead of editing the state before `weatherNode` in the `weatherGraph` subgraph, another way we could update the state is by acting as the `weatherNode` ourselves. We can do this by passing the subgraph config along with a node name passed as a third positional argument, which allows us to update the state as if we are the node we specify.

We will set an interrupt before the `weatherNode` and then using the update state function as the `weatherNode`, the graph itself never calls `weatherNode` directly but instead we decide what the output of `weatherNode` should be.

In [19]:
const streamWithAsNode = await graph.stream({
  messages: [{
    role: "user",
    content: "What's the weather in sf",
  }]
}, {
  configurable: {
    thread_id: "14",
  }
});

for await (const update of streamWithAsNode) {
  console.log(update);
}

// Graph execution should stop before the weather node
console.log("interrupted!");

const interruptedState = await graph.getState({
  configurable: {
    thread_id: "14",
  }
}, { subgraphs: true });

console.log(interruptedState);

// We update the state by passing in the message we want returned from the weather node
// and make sure to pass `"weatherNode"` to signify that we want to act as this node.
await graph.updateState((interruptedState.tasks[0].state as StateSnapshot).config, {
  messages: [{
    "role": "assistant",
    "content": "rainy"
  }]
}, "weatherNode");


const resumedStreamWithAsNode = await graph.stream(null, {
  configurable: {
    thread_id: "14",
  },
  streamMode: "updates",
  subgraphs: true,
});

for await (const update of resumedStreamWithAsNode) {
  console.log(update);
}

console.log(await graph.getState({
  configurable: {
    thread_id: "14",
  }
}, { subgraphs: true }));

{ routerNode: { route: 'weather' } }
interrupted!
{
  values: {
    messages: [
      HumanMessage {
        "content": "What's the weather in sf",
        "additional_kwargs": {},
        "response_metadata": {}
      }
    ],
    route: 'weather'
  },
  next: [ 'weatherGraph' ],
  tasks: [
    {
      id: 'f1342692-df24-5305-9b8c-34d6a8ab90f0',
      name: 'weatherGraph',
      path: [Array],
      interrupts: [],
      state: [Object]
    }
  ],
  metadata: {
    source: 'loop',
    writes: { routerNode: [Object] },
    step: 1,
    parents: {}
  },
  config: {
    configurable: {
      thread_id: '14',
      checkpoint_id: '1ef7c588-3051-6790-8001-0c24644b7dac',
      checkpoint_ns: ''
    }
  },
  createdAt: '2024-09-26T22:41:46.889Z',
  parentConfig: {
    configurable: {
      thread_id: '14',
      checkpoint_ns: '',
      checkpoint_id: '1ef7c588-2bce-6b02-8000-e95616f04495'
    }
  }
}
[ [], { weatherGraph: { messages: [Array] } } ]
{
  values: {
    messages: [
      HumanMe

Perfect! The agent responded with the message we passed in ourselves, and identified the weather in SF as `rainy` instead of `sunny`.

### Acting as the entire subgraph

Lastly, we could also update the graph just acting as the **entire** subgraph. This is similar to the case above but instead of acting as just the `weatherNode` we are acting as the entire `weatherGraph` subgraph. This is done by passing in the normal graph config as well as the `asNode` argument, where we specify the we are acting as the entire subgraph node.

In [20]:
const entireSubgraphExampleStream = await graph.stream({
  messages: [
    {
      role: "user",
      content: "what's the weather in sf"
    }
  ],
}, {
  configurable: {
    thread_id: "8",
  },
  streamMode: "updates",
  subgraphs: true,
});

for await (const update of entireSubgraphExampleStream) {
  console.log(update);
}

// Graph execution should stop before the weather node
console.log("interrupted!");

// We update the state by passing in the message we want returned from the weather graph.
// Note that we don't need to pass in the subgraph config, since we aren't updating the state inside the subgraph
await graph.updateState({
  configurable: {
    thread_id: "8",
  }
}, {
  messages: [{ role: "assistant", content: "rainy" }]
}, "weatherGraph");

const resumedEntireSubgraphExampleStream = await graph.stream(null, {
  configurable: {
    thread_id: "8",
  },
  streamMode: "updates",
});

for await (const update of resumedEntireSubgraphExampleStream) {
  console.log(update);
}

const currentStateAfterUpdate = await graph.getState({
  configurable: {
    thread_id: "8",
  }
});

console.log(currentStateAfterUpdate.values.messages);

[ [], { routerNode: { route: 'weather' } } ]
[
  [ 'weatherGraph:0f667666-5971-52cf-aa3c-f1f12f9426e6' ],
  { modelNode: { city: 'San Francisco' } }
]
interrupted!
[
  HumanMessage {
    "content": "what's the weather in sf",
    "additional_kwargs": {},
    "response_metadata": {}
  },
  AIMessage {
    "content": "rainy",
    "additional_kwargs": {},
    "response_metadata": {},
    "tool_calls": [],
    "invalid_tool_calls": []
  }
]


Again, the agent responded with "rainy" as we expected.

## Double nested subgraphs

This same functionality continues to work no matter the level of nesting. Here is an example of doing the same things with a double nested subgraph (although any level of nesting will work). We add another router on top of our already defined graphs.

First, let's recreate the graph we've been using above. This time we will compile it with no checkpointer, since it itself will be a subgraph!

In [21]:
const parentGraph = new StateGraph(RouterStateAnnotation)
  .addNode("routerNode", routerNode)
  .addNode("normalLLMNode", normalLLMNode)
  .addNode("weatherGraph", subgraph)
  .addEdge("__start__", "routerNode")
  .addConditionalEdges("routerNode", routeAfterPrediction)
  .addEdge("normalLLMNode", "__end__")
  .addEdge("weatherGraph", "__end__")
  .compile();

Now let's declare a "grandparent" graph that uses this graph as a subgraph:

In [22]:
const checkpointer = new MemorySaver();

const GrandfatherStateAnnotation = Annotation.Root({
  ...MessagesAnnotation.spec,
  toContinue: Annotation<boolean>,
});

const routerNode = async (_state: typeof GrandfatherStateAnnotation.State) => {
  // Dummy logic that will always continue
  return { toContinue: true };
};

const routeAfterPrediction = async (state: typeof GrandfatherStateAnnotation.State) => {
  if (state.toContinue) {
    return "parentGraph";
  } else {
    return "__end__";
  }
};

const grandparentGraph = new StateGraph(GrandfatherStateAnnotation)
  .addNode("routerNode", routerNode)
  .addNode("parentGraph", parentGraph)
  .addEdge("__start__", "routerNode")
  .addConditionalEdges("routerNode", routeAfterPrediction)
  .addEdge("parentGraph", "__end__")
  .compile({ checkpointer });

In [23]:
// from IPython.display import Image, display

// # Setting xray to 1 will show the internal structure of the nested graph
// display(Image(grandparent_graph.get_graph(xray=2).draw_mermaid_png()))

If we run until the interrupt, we can now see that there are snapshots of the state of all three graphs

In [24]:
const grandparentConfig = {
  configurable: { thread_id: "123" },
};

const grandparentGraphStream = await grandparentGraph.stream({
  messages: [{
    role: "user",
    content: "what's the weather in SF"
  }],
}, {
  ...grandparentConfig,
  streamMode: "updates",
  subgraphs: true
});

for await (const update of grandparentGraphStream) {
  console.log(update);
}

[ [], { routerNode: { toContinue: true } } ]
[
  [ 'parentGraph:b6af3181-3b4a-50e9-9140-ca58e201b842' ],
  { routerNode: { route: 'weather' } }
]
[
  [
    'parentGraph:b6af3181-3b4a-50e9-9140-ca58e201b842',
    'weatherGraph:b06d1b38-1f96-5e23-88fa-cb431a6626ef'
  ],
  { modelNode: { city: 'San Francisco' } }
]


In [26]:
const grandparentGraphState = await grandparentGraph.getState(
  grandparentConfig, 
  { subgraphs: true }
);

const parentGraphState = grandparentGraphState.tasks[0].state as StateSnapshot;
const subgraphState = parentGraphState.tasks[0].state as StateSnapshot;

console.log("Grandparent State:");
console.log(grandparentGraphState.values);
console.log("---------------");
console.log("Parent Graph State:");
console.log(parentGraphState.values);
console.log("---------------");
console.log("Subgraph State:");
console.log(subgraphState.values);

Grandparent State:
{
  messages: [
    HumanMessage {
      "content": "what's the weather in SF",
      "additional_kwargs": {},
      "response_metadata": {}
    }
  ],
  toContinue: true
}
---------------
Parent Graph State:
{
  messages: [
    HumanMessage {
      "content": "what's the weather in SF",
      "additional_kwargs": {},
      "response_metadata": {}
    }
  ],
  route: 'weather'
}
---------------
Subgraph State:
{
  messages: [
    HumanMessage {
      "content": "what's the weather in SF",
      "additional_kwargs": {},
      "response_metadata": {}
    }
  ],
  city: 'San Francisco'
}


We can now continue, acting as the node three levels down

In [27]:
await grandparentGraph.updateState(subgraphState.config, {
  messages: [{
    role: "assistant",
    content: "rainy"
  }]
}, "weatherNode");

const updatedGrandparentGraphStream = await grandparentGraph.stream(null, {
  ...grandparentConfig,
  streamMode: "updates",
  subgraphs: true,
});

for await (const update of updatedGrandparentGraphStream) {
  console.log(update);
}

console.log((await grandparentGraph.getState(grandparentConfig)).values.messages)

[
  [ 'parentGraph:b6af3181-3b4a-50e9-9140-ca58e201b842' ],
  { weatherGraph: { messages: [Array] } }
]
[ [], { parentGraph: { messages: [Array] } } ]
[
  HumanMessage {
    "content": "what's the weather in SF",
    "additional_kwargs": {},
    "response_metadata": {}
  },
  HumanMessage {
    "content": "what's the weather in SF",
    "additional_kwargs": {},
    "response_metadata": {}
  },
  HumanMessage {
    "content": "what's the weather in SF",
    "additional_kwargs": {},
    "response_metadata": {}
  },
  AIMessage {
    "content": "rainy",
    "additional_kwargs": {},
    "response_metadata": {},
    "tool_calls": [],
    "invalid_tool_calls": []
  }
]


As in the cases above, we can see that the AI responds with "rainy" as we expect.

We can explore the state history to see how the state of the grandparent graph was updated at each step.

In [28]:
const grandparentStateHistories = await grandparentGraph.getStateHistory(grandparentConfig);
for await (const state of grandparentStateHistories) {
  console.log(state);
  console.log("-----");
}

{
  values: {
    messages: [
      HumanMessage {
        "content": "what's the weather in SF",
        "additional_kwargs": {},
        "response_metadata": {}
      },
      HumanMessage {
        "content": "what's the weather in SF",
        "additional_kwargs": {},
        "response_metadata": {}
      },
      HumanMessage {
        "content": "what's the weather in SF",
        "additional_kwargs": {},
        "response_metadata": {}
      },
      AIMessage {
        "content": "rainy",
        "additional_kwargs": {},
        "response_metadata": {},
        "tool_calls": [],
        "invalid_tool_calls": []
      }
    ],
    toContinue: true
  },
  next: [],
  tasks: [],
  metadata: {
    source: 'loop',
    writes: { parentGraph: [Object] },
    step: 2,
    parents: {}
  },
  config: {
    configurable: {
      thread_id: '123',
      checkpoint_ns: '',
      checkpoint_id: '1ef7c589-122a-6db1-8002-5113be7f828c'
    }
  },
  createdAt: '2024-09-26T22:42:10.571Z',
  paren